In [1]:
# https://towardsdatascience.com/automating-portfolio-optimization-using-python-9f344b9380b9
# Mean varieance analysis
# What is the best method of carrying portfolio analysis, what other methods are available

In [2]:
#Importing all required libraries
#Created by Sanket Karve
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as web
from matplotlib.ticker import FuncFormatter

In [17]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
# from pypfopt.cla import CLA
from pypfopt import discrete_allocation
from pypfopt import plotting
from matplotlib.ticker import FuncFormatter

In [18]:
# Tickers from Boston Scientific, Berkshire Hathway, Invesco Trust, S&P Index Fund, AES Corp., and Sealed Air Corp
# Adjusted close value
tickers = ['BSX', 'AES', 'BRK-B', 'SEE', 'QQQ', 'SPY']
thelen = len(tickers)
price_data = []

for ticker in range(thelen):
    prices = web.DataReader(tickers[ticker], start='2015-01-01', end='2020-06-06', data_source='yahoo')
    price_data.append(prices.assign(ticker=ticker)[['Adj Close']])
    
df_stocks = pd.concat(price_data, axis=1)
df_stocks.columns=tickers
df_stocks.head()

,BSX,AES,BRK-B,SEE,QQQ,SPY
Date,,,,,,
2014-12-31,13.25,10.952408,150.149994,38.902077,97.966927,182.844467
2015-01-02,13.22,10.896731,149.169998,39.149628,97.672798,182.746567
2015-01-05,13.81,10.586535,147.000000,38.590351,96.240067,179.446259
2015-01-06,13.70,10.347920,146.839996,38.792053,94.949654,177.756073
2015-01-07,14.03,10.363826,148.880005,38.847069,96.173660,179.971115


In [19]:
nullin_df = pd.DataFrame(df_stocks, columns=tickers)
print(nullin_df.isnull().sum())

BSX      0
AES      0
BRK-B    0
SEE      0
QQQ      0
SPY      0
dtype: int64


In [36]:
# Annualised return
mu = expected_returns.mean_historical_return(df_stocks)

# Sample Variance Portfolio
Sigma = risk_models.sample_cov(df_stocks)
mu.head()


BSX      0.219392
AES      0.045931
BRK-B    0.054952
SEE     -0.019985
QQQ      0.178689
dtype: float64

In [33]:
Sigma.head()

,BSX,AES,BRK-B,SEE,QQQ,SPY
BSX,0.083120,0.040805,0.032322,0.037851,0.038963,0.035796
AES,0.040805,0.107820,0.033929,0.051781,0.032859,0.034929
BRK-B,0.032322,0.033929,0.042899,0.030697,0.032195,0.032527
SEE,0.037851,0.051781,0.030697,0.099864,0.027641,0.029476
QQQ,0.038963,0.032859,0.032195,0.027641,0.044236,0.036097


In [21]:
ef = EfficientFrontier(mu, Sigma, weight_bounds=(-1, 1))
sharpe_pfolio=ef.max_sharpe()
sharpe_pwt=ef.clean_weights()
print(sharpe_pwt)

OrderedDict([('BSX', 0.82969), ('AES', -0.14794), ('BRK-B', -0.92165), ('SEE', -0.46673), ('QQQ', 1.0), ('SPY', 0.70663)])


In [37]:
print(sharpe_pfolio)

OrderedDict([('BSX', 0.8296922846767448), ('AES', -0.1479446896133863), ('BRK-B', -0.9216466615631314), ('SEE', -0.4667327623046504), ('QQQ', 1.0), ('SPY', 0.7066318288044233)])


In [22]:
ef1 = EfficientFrontier(mu, Sigma, weight_bounds=(0, 1))
minvol = ef1.min_volatility()
minvol_pwt = ef1.clean_weights()
# print(sharpe_pwt)
print(minvol_pwt)


OrderedDict([('BSX', 0.0), ('AES', 0.0), ('BRK-B', 0.08013), ('SEE', 0.05241), ('QQQ', 0.0), ('SPY', 0.86746)])


In [38]:
latest_prices = discrete_allocation.get_latest_prices(df_stocks)

# Min volatility Portfolio Allocation=10000
allocation_minv, rem_minv = discrete_allocation.DiscreteAllocation(minvol_pwt, latest_prices,total_portfolio_value=10000).lp_portfolio()
print(allocation_minv)
# print(rem_minv)
print("Leftover Fund value in$ after building minimum volatility portfolio is ${:.2f}".format(rem_minv))
latest_prices.head()

{'BRK-B': 4, 'SEE': 16, 'SPY': 27}
Leftover Fund value in$ after building minimum volatility portfolio is $125.56


BSX       38.830002
AES       13.971002
BRK-B    200.660004
SEE       34.869690
QQQ      238.840988
Name: 2020-06-05 00:00:00, dtype: float64

In [29]:
#Max Sharpe Ratio Portfolio Allocation $10000
latest_prices1 = discrete_allocation.get_latest_prices(df_stocks)
allocation_shp, rem_shp = discrete_allocation.DiscreteAllocation(sharpe_pwt, latest_prices1, total_portfolio_value=1000).lp_portfolio() 
print(allocation_shp)
print("Leftover Fund value in$ after building Max Sharpe ratio portfolio is ${:.2f}".format(rem_shp))

#allocation using integer programming via PyPortfolioOpt User Guide
#Alex Putkov code used for guidance and reference in applying integer programming

{'BSX': 5, 'QQQ': 2, 'SPY': 1, 'AES': -8, 'BRK-B': -5, 'SEE': -12}
Leftover Fund value in$ after building Max Sharpe ratio portfolio is $15.65
